In [1]:
!pip install -qU langchain-experimental


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip -q install pypdf faiss-cpu sentence-transformers
!pip -q install langchain langchain-community langchain-text-splitters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 47.8 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import PyPDFLoader

pdf_paths = [
    "/content/CHAPTER I&II-Leadership.pdf",



    "/content/دقيقة في القيادة 1.pdf",
]

docs = []
for p in pdf_paths:
    loader = PyPDFLoader(p)
    loaded = loader.load()  # one Document per page (keeps page metadata)
    # Add file source explicitly (handy later)
    for d in loaded:
        d.metadata["source_file"] = p
    docs.extend(loaded)

len(docs), docs[0].metadata


(195,
 {'producer': 'macOS Version 12.0.1 (Build 21A559) Quartz PDFContext',
  'creator': 'PyPDF',
  'creationdate': "D:20260204133134Z00'00'",
  'moddate': "D:20260204133134Z00'00'",
  'source': '/content/CHAPTER I&II-Leadership.pdf',
  'total_pages': 73,
  'page': 0,
  'page_label': '1',
  'source_file': '/content/CHAPTER I&II-Leadership.pdf'})

In [4]:
import re

def clean_text(t: str) -> str:
    t = t.replace("\u00ad", "")  # soft hyphen
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\n{2,}", "\n\n", t)
    t = re.sub(r"(\w)-\n(\w)", r"\1\2", t)  # de-hyphenation across lines
    return t.strip()

for d in docs:
    d.page_content = clean_text(d.page_content)

# sanity check
print(docs[0].page_content[:800])


CHAPTER I: INTRODUCTION 1.1 Statement of the Problem Colleges and universities in Jordan and throughout the Middle East region perform a critical role and service in supporting and furthering the public good by educating their students and by contributing to and advancing the institutional aspirations of government agencies, industry, foundations and other sectors of society through their policies and best practices (Kezar, Carducci, & Contreras-McGavin, 2006). The higher education sector in Jordan and throughout the region is faced with a number of significant challenges including funding shortages; limited access to higher educational opportunities; the lack of standardized and competitive admissions systems; inadequate accreditation and quality assurance systems and ineffective governan


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings

EMBED_MODEL = "intfloat/multilingual-e5-large"

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

# E5 works best if you prefix texts (we'll do it automatically in a wrapper later if needed)


/tmp/ipython-input-80528909.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [6]:
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=90
)

chunks = text_splitter.split_documents(docs)

len(chunks)


345

In [7]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

SAVE_DIR = "/content/faiss_index_leadership"
vectorstore.save_local(SAVE_DIR)

print("Saved FAISS index to:", SAVE_DIR)


Saved FAISS index to: /content/faiss_index_leadership


In [21]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

SAVE_DIR = "/content/faiss_index_leadership"

EMBED_MODEL = "intfloat/multilingual-e5-large"  # must match what you used earlier
embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

vectorstore = FAISS.load_local(
    SAVE_DIR,
    embeddings,
    allow_dangerous_deserialization=True
)


Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

XLMRobertaModel LOAD REPORT from: intfloat/multilingual-e5-large
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [24]:
!pip install -q bitsandbytes
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

GEN_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"

# Define quantization configuration
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config, # Use the quantization config here
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.1 MB/s eta 0:00:00


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [25]:
def format_context(docs, max_chars=12000):
    blocks = []
    total = 0
    for d in docs:
        src = d.metadata.get("source_file", "unknown")
        page = d.metadata.get("page", "unknown")
        text = d.page_content.strip()
        block = f"[SOURCE: {src} | page: {page}]\n{text}"
        blocks.append(block)
        total += len(block)
        if total >= max_chars:
            break
    return "\n\n---\n\n".join(blocks)

SYSTEM_RULES = """You are a bilingual RAG assistant (Arabic + English).
Use ONLY the provided context to answer.
If the answer is not in the context, say you don't know.
Cite sources inline like: (SOURCE: filename | page: X).
Answer in the same language as the question.
"""


In [26]:
def mistral_generate(question: str, context: str, max_new_tokens=400):
    messages = [
        {"role": "system", "content": SYSTEM_RULES},
        {"role": "user", "content": f"QUESTION:\n{question}\n\nCONTEXT:\n{context}\n\nANSWER:"}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.3,
            top_p=0.95,
            repetition_penalty=1.1,
            eos_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # The decoded text includes the prompt; try to return only the answer tail:
    return decoded.split("ANSWER:")[-1].strip()


In [27]:
def rag_answer_hf(question: str, k: int = 5):
    retriever.search_kwargs = {"k": k}
    docs = retriever.invoke(question)   # NEW API

    context = format_context(docs)
    answer = mistral_generate(question, context)

    return answer, docs


In [28]:
answer, used_docs = rag_answer_hf("What are the key principles of leadership discussed here?")
print(answer)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[/INST] The key principles of leadership discussed in the study are:

1. Behaviors that define and lead others to a shared vision.
2. Instilling a sense of trust and teamwork.
3. Modeling and encouraging effective engagement.
4. Improving organizations through better processes.
5. Emphasizing mutual respect and accountability.

These principles align with the Transformational Leadership theory, which the study focuses on. (SOURCE: /content/CHAPTER I&II-Leadership.pdf | pages: 3, 71)


In [31]:
answer, used_docs = rag_answer_hf("ما هر تعريف القيادة")
print(answer)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[/INST] بعض التعريف للقيادة هناك بعض، ولكنها غير مقبولة شاملًا. بحسب منصورة واحدة من الشاعرين المتداولة في المستudies في القيادة ("Leadership is one of the most observed and least understood phenomena on earth" - SOURCE: /content/CHAPTER I&II-Leadership.pdf | page: 31), فإنها لا توجد نقاط تبديلة للتعريف الشامل لها. في بعض الأنحاء، القيادة تعني تأثيرنا على الأشخاص الذين نهددهم في الطريق الصحيح. وفي بعض الأحيان، تعني انتقاء وتوجيه بعض الأشخاص في أهدافنا وحلول المشاكل وبناء القدرات التنفسية للOrganization. فما هي القيادة وهي شيء جميل جداً ولا تناسب التعريف الش
